#Note:
- If Numaric Column contian <5% missing value and Randomly. Then you can use --->mean,median stratgy for handeling missing value

- but when not randomly use 'End of Distribution'--1)Q1-1.5IQR 2)Q1+1.5IQR where IQR=Q3-Q1

- When data is Numercal Catagorical and Missing value >10% .Then use Arbitary Missing value handeling or insert -1 where NaN value.
- When Data String Catagorical and Missing value >10% .Then use "missing/empty" string as a new Catagory  to NaN value



```
# si=SimpleImputer(stratgy='constant',fill_value=-1 | "missing/empty")
X_train['new']=si.fit_transfrom(X_train)
```



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('/content/Titanic-Dataset.csv')
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [ ]:
df=df.iloc[:,[1,2,4,5,6,7,9,10,11]]
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S


In [ ]:
df['Cabin'].isnull().sum()/df.shape[0]*100

np.float64(77.10437710437711)

In [ ]:
df.drop('Cabin',axis=1,inplace=True)
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer,MissingIndicator
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,LabelEncoder
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import FunctionTransformer,PowerTransformer
from sklearn.compose import ColumnTransformer,make_column_transformer
from sklearn.pipeline import Pipeline,make_pipeline
import scipy.stats as stats

from sklearn.linear_model import LogisticRegression,LinearRegression

#Casually Implement of MissingIndicator

In [ ]:
df.head(2)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C


In [ ]:
mi=MissingIndicator()
df['new_age']=mi.fit_transform(df[['Age']])
df.sample(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,new_age
246,0,3,female,25.0,0,0,7.7750,S,False
358,1,3,female,NaN,0,0,7.8792,Q,True
473,1,2,female,23.0,0,0,13.7917,C,False
280,0,3,male,65.0,0,0,7.7500,Q,False
807,0,3,female,18.0,0,0,7.7750,S,False
53,1,2,female,29.0,1,0,26.0000,S,False
785,0,3,male,25.0,0,0,7.2500,S,False
7,0,3,male,2.0,3,1,21.0750,S,False
279,1,3,female,35.0,1,1,20.2500,S,False
240,0,3,female,NaN,1,0,14.4542,C,True


In [ ]:
df.shape

(891, 9)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(['Survived','new_age'],axis=1),df['Survived'],test_size=0.2,random_state=42)
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [ ]:
age_pipe=Pipeline([
    ('imp_age',SimpleImputer(strategy='median'))
])
fare_pipe=Pipeline([
    ('normalize_fare',PowerTransformer(method='yeo-johnson'))
])

sex_emb_pipe=Pipeline([
    ('sex_imp',SimpleImputer(strategy='most_frequent')),
    ('ohe_sex_emb',OneHotEncoder(dtype=np.int32,drop='first',sparse_output=False,handle_unknown='ignore'))
])

In [ ]:
CT1=make_column_transformer(
    (age_pipe,['Age']),
    (fare_pipe,['Fare']),
    (sex_emb_pipe,['Sex','Embarked']),
    remainder='passthrough'
)

In [ ]:
CT1.fit_transform(X_train)

array([[45.5       ,  0.47999826,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [23.        , -0.28375264,  1.        , ...,  2.        ,
         0.        ,  0.        ],
       [32.        , -0.77244668,  1.        , ...,  3.        ,
         0.        ,  0.        ],
       ...,
       [41.        , -0.20313477,  1.        , ...,  3.        ,
         2.        ,  0.        ],
       [14.        ,  1.78386853,  0.        , ...,  1.        ,
         1.        ,  2.        ],
       [21.        ,  1.40043606,  1.        , ...,  1.        ,
         0.        ,  1.        ]])

In [ ]:
pipe_before=make_pipeline(CT1,StandardScaler(),LogisticRegression())

pipe_before.fit(X_train,y_train)


/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('imp_age',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Age']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('normalize_fare',
                                                                   PowerTransformer())]),
                                                  ['Fare']),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('sex_imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe_sex_emb',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'numpy.int32'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Sex', 'Embarked'])])),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
y_pred=pipe_before.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.7988826815642458

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe_before,X_train,y_train,cv=5,scoring='accuracy').mean()

np.float64(0.7864572047670639)

##after Missing Indicator(builtin and manually)

In [ ]:
age_p=Pipeline([
    # ('mi_age',MissingIndicator()),
    ('imu_age',SimpleImputer(add_indicator=True))
])

In [ ]:
CT2=make_column_transformer(
    (age_p,['Age']),
    (fare_pipe,['Fare']),
    (sex_emb_pipe,['Sex','Embarked']),
    remainder='passthrough'
)
CT2.fit_transform(X_train)

array([[45.5       ,  0.        ,  0.47999826, ...,  1.        ,
         0.        ,  0.        ],
       [23.        ,  0.        , -0.28375264, ...,  2.        ,
         0.        ,  0.        ],
       [32.        ,  0.        , -0.77244668, ...,  3.        ,
         0.        ,  0.        ],
       ...,
       [41.        ,  0.        , -0.20313477, ...,  3.        ,
         2.        ,  0.        ],
       [14.        ,  0.        ,  1.78386853, ...,  1.        ,
         1.        ,  2.        ],
       [21.        ,  0.        ,  1.40043606, ...,  1.        ,
         0.        ,  1.        ]])

In [ ]:
pipe_after=make_pipeline(CT2,StandardScaler(),LogisticRegression())
pipe_after.fit(X_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('imu_age',
                                                                   SimpleImputer(add_indicator=True))]),
                                                  ['Age']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('normalize_fare',
                                                                   PowerTransformer())]),
                                                  ['Fare']),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('sex_imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe_sex_emb',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'numpy.int32'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Sex', 'Embarked'])])),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
y_pred=pipe_after.predict(X_test)
accuracy_score(y_pred,y_test)

0.8100558659217877

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe_after,X_train,y_train,cv=5,scoring='accuracy').mean()

np.float64(0.7864572047670639)

#check manually

In [ ]:
df.head()
#True and False jodi bool type hoi tahole ML a Encode korte hobe na

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,new_age
0,0,3,male,22.0,1,0,7.2500,S,False
1,1,1,female,38.0,1,0,71.2833,C,False
2,1,3,female,26.0,0,0,7.9250,S,False
3,1,1,female,35.0,1,0,53.1000,S,False
4,0,3,male,35.0,0,0,8.0500,S,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
 8   new_age   891 non-null    bool   
dtypes: bool(1), float64(2), int64(4), object(2)
memory usage: 56.7+ KB


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(['Survived'],axis=1),df['Survived'],test_size=0.2,random_state=42)
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,new_age
331,1,male,45.5,0,0,28.5,S,False
733,2,male,23.0,0,0,13.0,S,False


In [ ]:
age_pipe2=Pipeline([
    ('age_imp',SimpleImputer())
])
fare_pipe2=Pipeline([
    ('normalization_fare',PowerTransformer())
])

sex_emb_pipe2=Pipeline([
    ('sex_imp',SimpleImputer(strategy='most_frequent')),
    ('ohe_sex_emb',OneHotEncoder(dtype=np.int32,drop='first',sparse_output=False,handle_unknown='ignore'))
])

CT3=make_column_transformer(
    (age_pipe2,['Age']),
    (fare_pipe2,['Fare']),
    (sex_emb_pipe2,['Sex','Embarked']),
    remainder='passthrough'
)

CT3.fit_transform(X_train)


array([[45.5, 0.47999826231989395, 1, ..., 0, 0, False],
       [23.0, -0.28375263784654847, 1, ..., 0, 0, False],
       [32.0, -0.7724466813890875, 1, ..., 0, 0, False],
       ...,
       [41.0, -0.20313476693661398, 1, ..., 2, 0, False],
       [14.0, 1.7838685328829773, 0, ..., 1, 2, False],
       [21.0, 1.400436058476949, 1, ..., 0, 1, False]], dtype=object)

In [ ]:
pipe3=make_pipeline(CT3,StandardScaler(),LogisticRegression())
pipe3.fit(X_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('pipeline-1',
                                                  Pipeline(steps=[('age_imp',
                                                                   SimpleImputer())]),
                                                  ['Age']),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('normalization_fare',
                                                                   PowerTransformer())]),
                                                  ['Fare']),
                                                 ('pipeline-3',
                                                  Pipeline(steps=[('sex_imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe_sex_emb',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'numpy.int32'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Sex', 'Embarked'])])),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
y_pred3=pipe3.predict(X_test)
accuracy_score(y_pred3,y_test)

0.8100558659217877

In [ ]:
cross_val_score(pipe3,X_train,y_train,cv=5,scoring='accuracy').mean()

np.float64(0.7864572047670639)

#Grid SearchCV: Find automatically better parameter for any Transformation which increase accuracy
[SEE Latter]

In [ ]:
# from sklearn.model_selection import GridSearchCV
# pram_grid={
#     'A':[0.01, 0.1, 1, 10],
#     'imp_age':['mean','median'],
#     'imp_cat':['most_frequent','constant']
# }
# grid_search=GridSearchCV(pipe3,pram_grid,cv=5,scoring='accuracy')
# grid_search.fit(X_train,y_train)

In [ ]:
# pram_grid = {
#     'logisticregression__C': [0.01, 0.1, 1, 10],  # Assuming 'A' was intended for LogisticRegression C
#     'columntransformer__pipeline-1__simpleimputer__strategy': ['mean','median'], # Strategy for SimpleImputer in age_pipe2
#     'columntransformer__pipeline-3__simpleimputer__strategy': ['most_frequent','constant'] # Strategy for SimpleImputer in sex_emb_pipe2
# }

# grid_search=GridSearchCV(pipe3, pram_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train,y_train)

#Day-39:KNN Imputer

In [ ]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,new_age
331,1,male,45.5,0,0,28.5000,S,False
733,2,male,23.0,0,0,13.0000,S,False
382,3,male,32.0,0,0,7.9250,S,False
704,3,male,26.0,1,0,7.8542,S,False
813,3,female,6.0,4,2,31.2750,S,False


In [ ]:
X_train['Age'].isnull().sum()

np.int64(140)

In [ ]:
#ChatGPT


from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# 1. Load Titanic dataset
df = df
X = df.drop('Survived', axis=1)
y = df['Survived']

# 2. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Select columns
numeric_cols = ['Age', 'Fare', 'SibSp', 'Parch']
categorical_cols = ['Sex', 'Embarked']

# 4. Step 1: Numeric imputation using KNN
knn_impute_pipe = Pipeline([
    ('knn_imputer', KNNImputer())
])

# 5. Step 2: PowerTransform only Fare
fare_power_pipe = Pipeline([
    ('power', PowerTransformer(method='yeo-johnson'))
])

# 6. Step 3: Categorical encode
cat_pipe = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 7. Combine: ColumnTransformer
preprocessor = ColumnTransformer([
    ('knn_num', knn_impute_pipe, numeric_cols),# shob golo column use kore just akta column "Age" ei value Add korse[Onno kono Column k Affact kore nai. Tai kaj korche]
                                              #Ta o jodi kaj na kore Tahole Fare Column k bad dia all Numeric column pass koro
    ('fare_power', fare_power_pipe, ['Fare']),
    ('cat', cat_pipe, categorical_cols)
])

# 8. Final Pipeline: Preprocessor + Model
model_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', LogisticRegression(max_iter=1000))
])

# 9. Train
model_pipeline.fit(X_train, y_train)

# 10. Predict & Evaluate
y_pred = model_pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.770949720670391


In [ ]:
from sklearn.impute import KNNImputer
knn_age_pipe=Pipeline([
    ('knn_age',KNNImputer()) #just Age colum Pass korle hobe na All Numerical Column Pass koro(with Encoded if have)
])

In [ ]:
CT4=ColumnTransformer([
    ('knn_age_imp',knn_age_pipe,['Age']),#just Age ar opor KNN apply kore Fill kore.But KNN All column ar distance ar opor bitte kore value ber korer kotha
    ('normalize_fare',fare_pipe,['Fare']),
    ('sex_emb_ohe',sex_emb_pipe,['Sex','Embarked']),
],remainder='passthrough')

In [ ]:
pipe_knn=make_pipeline(
    CT4,StandardScaler(),LogisticRegression()
)
pipe_knn.fit(X_train,y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/compose/_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('knn_age_imp',
                                                  Pipeline(steps=[('knn_age',
                                                                   KNNImputer())]),
                                                  ['Age']),
                                                 ('normalize_fare',
                                                  Pipeline(steps=[('normalize_fare',
                                                                   PowerTransformer())]),
                                                  ['Fare']),
                                                 ('sex_emb_ohe',
                                                  Pipeline(steps=[('sex_imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe_sex_emb',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'numpy.int32'>,
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Sex', 'Embarked'])])),
                ('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [ ]:
y_pred_knn=pipe_knn.predict(X_test)
accuracy_score(y_pred_knn,y_test)

0.8100558659217877

In [ ]:
cross_val_score(pipe_knn,X_train,y_train,cv=5,scoring='accuracy').mean()

np.float64(0.7864572047670639)

#IterativeImputer:(use Lineare Model)

Iterative Imputer হল একটি স্মার্ট টেকনিক যেটা মিসিং ভ্যালু ফিল করতে মেশিন লার্নিং মডেল ব্যবহার করে। এটি একটার পর একটা ফিচার প্রেডিক্ট করে মিসিং ডেটা ফিলআপ করে।

###✅ কীভাবে কাজ করে?
IterativeImputer একে একে প্রতিটি ফিচারকে টার্গেট ধরে এবং বাকি ফিচারগুলোকে ইনপুট হিসেবে ব্যবহার করে মিসিং ভ্যালু প্রেডিক্ট করে। এটা একধরনের multivariate imputation।



**Missing value handling using Iterative Imputer** (from `sklearn.impute`) হল একটি উন্নত মিসিং ভ্যালু ইম্পিউটেশন টেকনিক যেখানে মডেল ব্যবহার করে একাধিক ফিচার থেকে একটি ফিচারের মিসিং ভ্যালু প্রেডিক্ট করা হয়।

---

### ✅ কীভাবে কাজ করে?

`IterativeImputer` একে একে প্রতিটি ফিচারকে টার্গেট ধরে এবং বাকি ফিচারগুলোকে ইনপুট হিসেবে ব্যবহার করে মিসিং ভ্যালু প্রেডিক্ট করে। এটা একধরনের **multivariate imputation**।

---

### 🔧 ব্যাবহার: Python (Scikit-learn)

```python
import pandas as pd
import numpy as np
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge  # বা অন্য রিগ্রেশন মডেল

# উদাহরণ ডেটাসেট
data = {
    'age': [25, np.nan, 35, 40, np.nan],
    'salary': [50000, 60000, np.nan, 80000, 75000],
    'experience': [2, 4, 5, np.nan, 3]
}
df = pd.DataFrame(data)

# ইম্পিউটার তৈরি
imputer = IterativeImputer(estimator=BayesianRidge(), max_iter=10, random_state=0)

# ফিট ও ট্রান্সফর্ম
df_imputed = imputer.fit_transform(df)

# রেজাল্ট DataFrame বানানো
df_imputed = pd.DataFrame(df_imputed, columns=df.columns)

print(df_imputed)
```

---

### 🔍 Parameters

* `estimator`: কোন মডেল দিয়ে ইম্পিউট করবেন (BayesianRidge, DecisionTreeRegressor, KNN ইত্যাদি)
* `max_iter`: কতবার ইটারেট করে ইম্পিউট করবে
* `initial_strategy`: শুরুতে মিসিং ভ্যালু কীভাবে প্রাথমিকভাবে ফিল করবে (mean, median, most\_frequent)
* `random_state` :Reproducivility জন্য random_state ইউজ করুন

---

### ✅ কবে ব্যবহার করবেন?

* যখন ডেটায় কম্প্লেক্স প্যাটার্ন থাকে
* যখন মাল্টিপল ফিচার একে অপরের উপর ডিপেন্ড করে or কোরিলেটেড
* Simple mean/median দিয়ে সঠিক ইম্পিউট করা যাচ্ছে না
* স্মল ডেটাসেটে ভালো কাজ করে (বিগ ডেটার জন্য SimpleImputer ফাস্টার)
---

### ⚠️ সাবধানতা:

* ধীর (slow) হতে পারে বড় ডেটাসেটে
* সবসময় স্কেল করে ব্যবহার করা উচিত (StandardScaler)
* High Memory Consumed in for Server(deploy with main dataset for user missing value handeling.
* Overfitting: খুব বেশি Iteration করলে ট্রেনিং ডেটায় ওভারফিট হতে পারে





In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer  # 👈 এইটা আগে থাকতে হবে
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# 1. Load Titanic dataset
df = df
X = df.drop('Survived', axis=1)
y = df['Survived']

# 2. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Select columns
numeric_cols = ['Age', 'Fare', 'SibSp', 'Parch']
categorical_cols = ['Sex', 'Embarked']

# 4. Step 1: Numeric imputation using IterativeImputer
iter_impute_pipe = Pipeline([
    ('iter_imputer', IterativeImputer(random_state=42))
])

# 5. Step 2: PowerTransform only Fare
fare_power_pipe = Pipeline([
    ('power', PowerTransformer(method='yeo-johnson'))
])

# 6. Step 3: Categorical encode
cat_pipe = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 7. Combine: ColumnTransformer
preprocessor = ColumnTransformer([
    ('iter_num', iter_impute_pipe, numeric_cols),
    ('fare_power', fare_power_pipe, ['Fare']),
    ('cat', cat_pipe, categorical_cols)
])

# 8. Final Pipeline: Preprocessor + Model
model_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', LogisticRegression(max_iter=1000))
])

# 9. Train
model_pipeline.fit(X_train, y_train)

# 10. Predict & Evaluate
y_pred = model_pipeline.predict(X_test)
print("Accuracy with Iterative Imputer:", accuracy_score(y_test, y_pred))


Accuracy with Iterative Imputer: 0.770949720670391


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import PowerTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Load Titanic dataset
# Make sure df is already loaded properly with 'Survived' column
X = df.drop('Survived', axis=1)
y = df['Survived']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Column groups
numeric_cols = ['Age', 'SibSp', 'Parch']         # Only these go to IterativeImputer
fare_col = ['Fare']                              # Separate transformer
categorical_cols = ['Sex', 'Embarked']

# Pipelines
iter_impute_pipe = Pipeline([
    ('iter_imputer', IterativeImputer(estimator=LogisticRegression(), random_state=42))
])

fare_power_pipe = Pipeline([
    ('power', PowerTransformer(method='yeo-johnson'))
])

cat_pipe = Pipeline([
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# ColumnTransformer
preprocessor = ColumnTransformer([
    ('iter_num', iter_impute_pipe, numeric_cols),
    ('fare_power', fare_power_pipe, fare_col),
    ('cat', cat_pipe, categorical_cols)
])

# Final pipeline
model_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('clf', LogisticRegression(max_iter=1000))
])

# Train
model_pipeline.fit(X_train, y_train)

# Predict & Evaluate
y_pred = model_pipeline.predict(X_test)
print("Accuracy with Iterative Imputer:", accuracy_score(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.